In [1]:
# Web scraping
import requests
from bs4 import BeautifulSoup
import pickle

articles = {}
# Article challenges
page = requests.get("https://www.challenges.fr/automobile/actu-auto/pres-de-150-000-precommandes-pour-le-cybertruck-de-tesla-selon-elon-musk_686441").text
soup = BeautifulSoup(page, "lxml")
headline = [p.text for p in soup.find(class_="article-start").find_all('h1')]
text = [p.text for p in soup.find(class_="corps").find_all('p')]
articles[0] = headline + text

In [2]:
# Articles leparisien
page = requests.get("https://www.leparisien.fr/economie/tesla-devoile-un-pick-up-electrique-sorti-tout-droit-d-un-film-de-science-fiction-22-11-2019-8199081.php").text
soup = BeautifulSoup(page, "lxml")
headline = [p.text for p in soup.find(class_="article_header").find_all('h1')] + [p.text for p in soup.find(class_="article_header").find_all('h2')]
text = [p.text for p in soup.find(class_="article-section margin_bottom_article").find_all('section')]
articles[1] = headline + text

In [3]:
journaux = ["challenges", "leparisien"]

In [4]:
for i, j in enumerate(journaux):
    with open("articles/" + j + ".txt", "wb") as file:
            pickle.dump(articles[i], file)

In [5]:
data_articles = {}
for i, j in enumerate(journaux):
    with open("articles/" + j + ".txt", "rb") as file:
        data_articles[j] = pickle.load(file) 

In [6]:
data_articles.keys()

dict_keys(['challenges', 'leparisien'])

In [7]:
data_articles["challenges"][:2]

['Près de 150.000 précommandes pour le Cybertruck de Tesla selon Elon Musk',
 'Lors de ce show jeudi à Los Angeles, un collaborateur de la firme avait lancé une masse en acier contre une vitre blindée de ce "Cybertruck" qui s\'était enfoncée sous le coup, arrachant un juron à Elon Musk. Une deuxième tentative avait endommagé une autre vitre.']

In [8]:
def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

In [9]:
data_combined = {key: [combine_text(value)] for (key, value) in data_articles.items()}

In [11]:
import pandas as pd

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ["text"]
data_df

,text
challenges,Près de 150.000 précommandes pour le Cybertruc...
leparisien,Tesla dévoile un pick-up électrique sorti tout...


In [12]:
data_df.text.loc["challenges"]

'Près de 150.000 précommandes pour le Cybertruck de Tesla selon Elon Musk Lors de ce show jeudi à Los Angeles, un collaborateur de la firme avait lancé une masse en acier contre une vitre blindée de ce "Cybertruck" qui s\'était enfoncée sous le coup, arrachant un juron à Elon Musk. Une deuxième tentative avait endommagé une autre vitre. Vendredi, les cours de l\'action Tesla avaient chuté de 6,1% après ce couac et des commentaires tièdes sur ce véhicule à venir. Mais samedi, Elon Musk a claironné sur Twitter: "Déjà 146.000 commandes de Cybertruck, 42% pour un moteur double, 41% pour un triple et 17% pour un moteur simple". Tout cela alors que le véhicule ne fait l\'objet d\'"aucune campagne de publicité ni de pré-paiements", a ajouté le milliardaire également PDG de SpaceX. Sur son site internet, Tesla offre la possiblité de passer commande de l\'un des trois modèles disponibles en échange d\'un dépôt symbolique "et entièrement remboursable" de 100 dollars. Il est donc impossible de di

In [18]:
import re
import string

def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

ct_1 = lambda x : clean_text(x)

In [19]:
data_clean = pd.DataFrame(data_df.text.apply(ct_1))

In [23]:
data_clean.text.loc["challenges"]

'près de  précommandes pour le cybertruck de tesla selon elon musk lors de ce show jeudi à los angeles un collaborateur de la firme avait lancé une masse en acier contre une vitre blindée de ce cybertruck qui sétait enfoncée sous le coup arrachant un juron à elon musk une deuxième tentative avait endommagé une autre vitre vendredi les cours de laction tesla avaient chuté de  après ce couac et des commentaires tièdes sur ce véhicule à venir mais samedi elon musk a claironné sur twitter déjà  commandes de cybertruck  pour un moteur double  pour un triple et  pour un moteur simple tout cela alors que le véhicule ne fait lobjet daucune campagne de publicité ni de prépaiements a ajouté le milliardaire également pdg de spacex sur son site internet tesla offre la possiblité de passer commande de lun des trois modèles disponibles en échange dun dépôt symbolique et entièrement remboursable de  dollars il est donc impossible de dire combien de ces réservations seront transformées en commandes fe

In [22]:
# Garder le texte originial
data_df.to_pickle("pickle/articles_originaux.pkl")

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

cv = CountVectorizer(stop_words=stopwords.words('french'))
data_cv = cv.fit_transform(data_clean.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexandre/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,accueillir,accélération,acier,adorer,affiche,ailleurs,ajouté,alors,américains,angeles,...,véhicules,véritable,zéro,échange,éclater,également,électrique,électriques,équipe,être
challenges,0,0,1,0,0,0,1,1,0,1,...,0,0,0,1,0,1,0,0,0,0
leparisien,1,1,1,1,1,1,0,0,1,1,...,1,1,1,0,1,0,2,1,1,1


In [30]:
data_dtm.to_pickle("pickle/dtm.pkl")
data_clean.to_pickle("pickle/articles_clean.pkl")